# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [22]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [23]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv", index_col = "City_ID")

# Display sample data
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
0,blackmans bay,-43.0167,147.3167,11.07,72,86,0.89,AU,2024-11-09 21:44:09
1,margate,51.3813,1.3862,8.16,87,92,3.16,GB,2024-11-09 21:45:17
2,zhezqazghan,47.7833,67.7667,-2.34,86,100,9.90,KZ,2024-11-09 21:46:45
3,adamstown,-25.0660,-130.1015,23.13,88,99,9.13,PN,2024-11-09 21:46:26
4,port-aux-francais,-49.3500,70.2167,5.04,90,81,9.98,TF,2024-11-09 21:44:55


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [24]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    size = "Humidity",
    color = "City",
    alpha = 0.5,
    geo = True,
    tiles = "OSM",

)


# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [25]:
# Narrow down cities that fit criteria and drop any results with null values
new_city_data_df = city_data_df[
(city_data_df["Max Temp"] > 21) & city_data_df["Max Temp"] < 27 &
(city_data_df["Wind Speed"] < 4.5) &
(city_data_df["Cloudiness"] == 0)
]

# Drop any rows with null values
new_city_data_df = new_city_data_df.dropna()

# Display sample data
new_city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
22,bethel,41.3712,-73.4140,9.76,35,0,0.89,US,2024-11-09 21:47:09
58,albany,42.6001,-73.9662,7.54,47,0,2.68,US,2024-11-09 21:46:19
65,tura,25.5198,90.2201,18.96,81,0,1.73,IN,2024-11-09 21:48:01
82,metlili chaamba,32.2667,3.6333,20.52,46,0,3.09,DZ,2024-11-09 21:48:27
91,swift current,50.2834,-107.8014,9.45,66,0,3.09,CA,2024-11-09 21:44:58


### Step 3: Create a new DataFrame called `hotel_df`.

In [26]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = new_city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
City_ID,,,,,,
22,bethel,US,41.3712,-73.4140,35,
58,albany,US,42.6001,-73.9662,47,
65,tura,IN,25.5198,90.2201,81,
82,metlili chaamba,DZ,32.2667,3.6333,46,
91,swift current,CA,50.2834,-107.8014,66,
104,fortuna,US,40.5982,-124.1573,52,
147,fort bragg,US,35.1390,-79.0060,44,
205,badger,US,64.8000,-147.5333,83,
206,rupert,US,42.6191,-113.6772,35,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [28]:
# Set parameters to search for a hotel

radius = 10000

params = {
    "categories" : "accommodation.hotel",
    "limit" : 1, 
    "api_keys":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = ["Lat"]
    lng = ["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params = params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bethel - nearest hotel: No hotel found
albany - nearest hotel: No hotel found
tura - nearest hotel: No hotel found
metlili chaamba - nearest hotel: No hotel found
swift current - nearest hotel: No hotel found
fortuna - nearest hotel: No hotel found
fort bragg - nearest hotel: No hotel found
badger - nearest hotel: No hotel found
rupert - nearest hotel: No hotel found
nagqu - nearest hotel: No hotel found
jamestown - nearest hotel: No hotel found
dali - nearest hotel: No hotel found
geraldton - nearest hotel: No hotel found
westport - nearest hotel: No hotel found
undurkhaan - nearest hotel: No hotel found
cidreira - nearest hotel: No hotel found
numaligarh - nearest hotel: No hotel found
safford - nearest hotel: No hotel found
baker city - nearest hotel: No hotel found
rio grande - nearest hotel: No hotel found
roma - nearest hotel: No hotel found
al badari - nearest hotel: No hotel found
dawson creek - nearest hotel: No hotel found
rovaniemi - nearest hotel: No h

,City,Country,Lat,Lng,Humidity,Hotel Name
City_ID,,,,,,
22,bethel,US,41.3712,-73.4140,35,No hotel found
58,albany,US,42.6001,-73.9662,47,No hotel found
65,tura,IN,25.5198,90.2201,81,No hotel found
82,metlili chaamba,DZ,32.2667,3.6333,46,No hotel found
91,swift current,CA,50.2834,-107.8014,66,No hotel found
104,fortuna,US,40.5982,-124.1573,52,No hotel found
147,fort bragg,US,35.1390,-79.0060,44,No hotel found
205,badger,US,64.8000,-147.5333,83,No hotel found
206,rupert,US,42.6191,-113.6772,35,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [29]:
%%capture --no-display

# Configure the map plot
hotelmap_plot = hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
    color = "Hotel Name",
    alpha = 0.5,
    geo = True,
    tiles = "OSM", 
    
    
)


# Display the map
hotelmap_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Hotel Name)